In [1]:
import pandas as pd
import sqlite3

# Load the CSV file
df = pd.read_csv("telco_churn.csv")

df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
# Create a SQLite database
conn = sqlite3.connect("churn.db")

conn


In [3]:
# Write DataFrame to SQL table
df.to_sql("customers", conn, if_exists="replace", index=False)

print("Table created successfully")


Table created successfully


In [4]:
pd.read_sql("SELECT COUNT(*) FROM customers;", conn)


,COUNT(*)
0,7043


In [5]:
query = """
SELECT Churn, COUNT(*) AS customer_count
FROM customers
GROUP BY Churn;
"""
pd.read_sql(query, conn)


,Churn,customer_count
0,No,5174
1,Yes,1869


In [6]:
query = """
SELECT
  Churn,
  COUNT(*) * 100.0 / (SELECT COUNT(*) FROM customers) AS churn_percentage
FROM customers
GROUP BY Churn;
"""
pd.read_sql(query, conn)


,Churn,churn_percentage
0,No,73.463013
1,Yes,26.536987


In [7]:
query = """
SELECT Contract,
       COUNT(*) AS total_customers,
       SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers
FROM customers
GROUP BY Contract;
"""
pd.read_sql(query, conn)


,Contract,total_customers,churned_customers
0,Month-to-month,3875,1655
1,One year,1473,166
2,Two year,1695,48


In [8]:
query = """
SELECT Contract,
       ROUND(
         100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
         2
       ) AS churn_rate_percentage
FROM customers
GROUP BY Contract;
"""
pd.read_sql(query, conn)


,Contract,churn_rate_percentage
0,Month-to-month,42.71
1,One year,11.27
2,Two year,2.83


In [9]:
query = """
SELECT COUNT(*) AS high_risk_customers
FROM customers
WHERE tenure < 12
  AND MonthlyCharges > 70;
"""
pd.read_sql(query, conn)


,high_risk_customers
0,826


In [10]:
query = """
SELECT
  Churn,
  COUNT(*) * 100.0 / (
    SELECT COUNT(*)
    FROM customers
    WHERE tenure < 12 AND MonthlyCharges > 70
  ) AS percentage
FROM customers
WHERE tenure < 12 AND MonthlyCharges > 70
GROUP BY Churn;
"""
pd.read_sql(query, conn)


,Churn,percentage
0,No,31.355932
1,Yes,68.644068
